In [11]:
import pandas as pd

df2 = pd.read_csv('Churn_Modelling2.csv')
dense2 = pd.read_csv('C_dense_output1.csv')
print("Read {} rows.".format(len(df2)))
print("Read {} rows.".format(len(dense2)))

# df = df.sample(frac=0.1, replace=False) # Uncomment this line to sample only 10% of the dataset

df2.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)
dense2.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)


# display 5 rows
df2[0:5]


Read 10000 rows.
Read 10000 rows.


,Geography,Gender,Age,Tenure,Balance,Exited
0,France,Female,42,2,0.00,1
1,Spain,Female,41,1,83807.86,0
2,France,Female,42,8,159660.80,1
3,France,Female,39,1,0.00,0
4,Spain,Female,43,2,125510.82,0


In [12]:
dense2[0:5]

,C_output_neuron_1,C_output_neuron_2
0,-0.783174,-0.326205
1,-0.606504,-0.440014
2,-0.995835,-1.536717
3,0.144759,0.501496
4,0.652626,2.063781


In [13]:
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None):
    if mean is None:
        mean = df[name].mean()

    if sd is None:
        sd = df[name].std()

    df[name] = (df[name] - mean) / sd

    
# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)

In [14]:
encode_text_dummy(df2, 'Geography')
encode_text_dummy(df2, 'Gender')
encode_numeric_zscore(df2, 'Age')
encode_numeric_zscore(df2, 'Tenure')
encode_numeric_zscore(df2, 'Balance')

In [15]:
df2[0:5]

,Age,Tenure,Balance,Exited,Geography-France,Geography-Germany,Geography-Spain,Gender-Female,Gender-Male
0,0.293503,-1.041708,-1.225786,1,1,0,0,1,0
1,0.198154,-1.387468,0.117344,0,0,0,1,1,0
2,0.293503,1.032856,1.332987,1,1,0,0,1,0
3,0.007456,-1.387468,-1.225786,0,1,0,0,1,0
4,0.388852,-1.041708,0.785689,0,0,0,1,1,0


In [16]:
merged_df2 = pd.concat([dense2, df2], axis=1)

In [17]:
merged_df2[0:5]

,C_output_neuron_1,C_output_neuron_2,Age,Tenure,Balance,Exited,Geography-France,Geography-Germany,Geography-Spain,Gender-Female,Gender-Male
0,-0.783174,-0.326205,0.293503,-1.041708,-1.225786,1,1,0,0,1,0
1,-0.606504,-0.440014,0.198154,-1.387468,0.117344,0,0,0,1,1,0
2,-0.995835,-1.536717,0.293503,1.032856,1.332987,1,1,0,0,1,0
3,0.144759,0.501496,0.007456,-1.387468,-1.225786,0,1,0,0,1,0
4,0.652626,2.063781,0.388852,-1.041708,0.785689,0,0,0,1,1,0


In [18]:
merged_df2.dropna(inplace=True,axis=1) # For now, just drop NA's (rows with missing values)

In [19]:
merged_df2[0:5]

,C_output_neuron_1,C_output_neuron_2,Age,Tenure,Balance,Exited,Geography-France,Geography-Germany,Geography-Spain,Gender-Female,Gender-Male
0,-0.783174,-0.326205,0.293503,-1.041708,-1.225786,1,1,0,0,1,0
1,-0.606504,-0.440014,0.198154,-1.387468,0.117344,0,0,0,1,1,0
2,-0.995835,-1.536717,0.293503,1.032856,1.332987,1,1,0,0,1,0
3,0.144759,0.501496,0.007456,-1.387468,-1.225786,0,1,0,0,1,0
4,0.652626,2.063781,0.388852,-1.041708,0.785689,0,0,0,1,1,0


In [21]:
# Convert to numpy - Classification
x_columns2 = merged_df2.columns.drop('Exited')
x2 = merged_df2[x_columns2].values
dummies2 = pd.get_dummies(merged_df2['Exited']) # Classification
outcomes2 = dummies2.columns
num_classes2 = len(outcomes2)
y2 = dummies2.values

In [20]:
merged_df2.groupby('Exited')['Exited'].count()

Exited
0    7963
1    2037
Name: Exited, dtype: int64

In [22]:
from sklearn.model_selection import train_test_split

from sklearn import metrics
from IPython.display import display, HTML 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation


x2_train, x2_test, y2_train, y2_test = train_test_split(
    x2, y2, test_size=0.25, random_state=42)

model = Sequential()
model.add(Dense(25, input_dim=x2.shape[1], activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(x2.shape[1])) # Multiple output neurons
model.compile(loss='mean_squared_error', optimizer='adam')


# Convert the output into a Pandas dataframe
pd2 = pd.DataFrame(x2, columns=['C_output_neuron_{}'.format(i+1) for i in range(x2.shape[1])])

# Save the output to a CSV file
pd2.to_csv('C_dense_output2.csv', index=False)

2023-04-17 14:11:23.748893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-17 14:11:25.805318: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()
model.add(Dense(10, input_dim=x2.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x2.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x2.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y2.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
model.fit(x2_train,y2_train,validation_data=(x2_test,y2_test),callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
235/235 - 1s - loss: 0.5067 - val_loss: 0.4294 - 508ms/epoch - 2ms/step
Epoch 2/1000
235/235 - 0s - loss: 0.4395 - val_loss: 0.4278 - 191ms/epoch - 814us/step
Epoch 3/1000
235/235 - 0s - loss: 0.4376 - val_loss: 0.4247 - 188ms/epoch - 798us/step
Epoch 4/1000
235/235 - 0s - loss: 0.4360 - val_loss: 0.4222 - 190ms/epoch - 807us/step
Epoch 5/1000
235/235 - 0s - loss: 0.4355 - val_loss: 0.4204 - 187ms/epoch - 794us/step
Epoch 6/1000
235/235 - 0s - loss: 0.4336 - val_loss: 0.4202 - 187ms/epoch - 794us/step
Epoch 7/1000
235/235 - 0s - loss: 0.4330 - val_loss: 0.4186 - 187ms/epoch - 798us/step
Epoch 8/1000
235/235 - 0s - loss: 0.4310 - val_loss: 0.4183 - 188ms/epoch - 801us/step
Epoch 9/1000
235/235 - 0s - loss: 0.4294 - val_loss: 0.4162 - 190ms/epoch - 809us/step
Epoch 10/1000
235/235 - 0s - loss: 0.4278 - val_loss: 0.4157 - 187ms/epoch - 795us/step
Epoch 11/1000
235/235 - 0s - loss: 0.4264 - val_loss: 0.4142 - 187ms/epoch - 794us/step
Epoch 12/1000
235/235 - 0s - loss: 0.4257 -

In [25]:
import numpy as np

# Measure accuracy
pred = model.predict(x2_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y2_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

79/79 [==============================] - 0s 507us/step
Validation score: 0.8184
